# Hopfield Network Experiments

This notebook showcases experimental results of Hopfield networks on the MNIST dataset, including pattern storage, recovery, and robustness tests under noise interference.

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch

# Add project root directory to path
sys.path.append('..')

# Import project modules
from src.hopfield.network import HopfieldNetwork
from src.hopfield.visualizer import HopfieldVisualizer
from src.hopfield.experiments import HopfieldExperiments
from src.utils.data_loader import MNISTLoader
from src.utils.preprocessing import binary_to_image
import config

## 1. Data Loading and Preprocessing

In [ ]:
# Create data loader
loader = MNISTLoader()

# Get samples of specific digits
digits = [0, 1, 2, 3, 4]
patterns = loader.get_specific_digit_samples(
    digits=digits, 
    num_samples=config.HOPFIELD_CONFIG['num_patterns'],
    binary_values={-1, 1}
)

# Visualize original patterns
fig, axes = plt.subplots(1, len(patterns), figsize=(15, 3))
for i, pattern in enumerate(patterns):
    img = binary_to_image(pattern.numpy(), (28, 28))
    axes[i].imshow(img, cmap='binary')
    axes[i].set_title(f'Digit {digits[i]}')
    axes[i].axis('off')
plt.tight_layout()
plt.show()

## 2. Hopfield Network Training

In [ ]:
# Create Hopfield network
network = HopfieldNetwork(n_neurons=28*28)

# Train the network
print(f"Training Hopfield network with {len(patterns)} patterns...")
network.train(patterns)
print("Training completed!")

## 3. Pattern Recovery Experiments

In [ ]:
# Create visualizer
visualizer = HopfieldVisualizer(network)

# Test pattern recovery
for i, pattern in enumerate(patterns):
    # Add noise
    noise_level = 0.2
    noisy_pattern = loader.add_noise(pattern, noise_level, binary_values={-1, 1})
    
    # Recover pattern
    recovered_pattern, energy_history = network.recover(
        noisy_pattern, 
        max_iterations=config.HOPFIELD_CONFIG['max_iterations'],
        mode='asynchronous'
    )
    
    # Calculate similarity
    similarity = network.calculate_similarity(pattern, recovered_pattern)
    
    print(f"Pattern {i+1} (Digit {digits[i]}): Original vs Recovered similarity = {similarity:.4f}")
    
    # Visualize recovery process
    visualizer.plot_recovery_comparison(
        pattern, noisy_pattern, recovered_pattern
    )

## 4. Energy Function Analysis

In [ ]:
# Plot energy convergence process
for i, pattern in enumerate(patterns):
    # Add noise
    noise_level = 0.2
    noisy_pattern = loader.add_noise(pattern, noise_level, binary_values={-1, 1})
    
    # Recover pattern and record energy history
    recovered_pattern, energy_history = network.recover(
        noisy_pattern, 
        max_iterations=config.HOPFIELD_CONFIG['max_iterations'],
        mode='asynchronous'
    )
    
    # Plot energy convergence process
    visualizer.plot_energy_convergence(energy_history, title=f'Energy Convergence for Digit {digits[i]}')

## 5. Weight Matrix Analysis

In [ ]:
# Visualize weight matrix
visualizer.plot_weight_matrix()

# Analyze eigenvalues of weight matrix
visualizer.plot_weight_eigenvalues()

## 6. Capacity Analysis Experiments

In [ ]:
# Create experiment object
experiments = HopfieldExperiments()

# Run capacity analysis experiment
capacity_results = experiments.capacity_analysis(max_patterns=20, noise_level=0.1)

# Visualize capacity analysis results
experiments.visualizer.plot_capacity_analysis(capacity_results)

## 7. Noise Interference Experiments

In [ ]:
# Run noise interference experiment
noise_results = experiments.noise_robustness_test(noise_levels=np.linspace(0, 0.5, 6))

# Visualize noise interference experiment results
experiments.visualizer.plot_noise_robustness(noise_results)

## 8. Attractor Analysis

In [ ]:
# Run attractor analysis experiment
attractor_results = experiments.attractor_analysis(n_random_starts=100)

# Visualize attractor analysis results
experiments.visualizer.plot_attractor_analysis(attractor_results)

## 9. 2D Energy Landscape Visualization

In [ ]:
# Visualize 2D energy landscape
for i, pattern in enumerate(patterns[:2]):  # Only visualize the first two patterns
    visualizer.plot_energy_landscape_2d(pattern, title=f'Energy Landscape for Digit {digits[i]}')

## 10. Confusion Matrix Analysis

In [ ]:
# Run confusion matrix analysis
confusion_results = experiments.confusion_matrix_analysis(noise_level=0.2)

# Visualize confusion matrix
experiments.visualizer.plot_confusion_matrix(confusion_results)

## Summary

This notebook showcases various experimental results of Hopfield networks on the MNIST dataset, including:

1. Pattern storage and recovery capabilities
2. Robustness under noise interference
3. Energy function convergence process
4. Weight matrix analysis
5. Network capacity analysis
6. Attractor analysis
7. 2D energy landscape visualization
8. Confusion matrix analysis

These experiments help us understand the working principles and performance characteristics of Hopfield networks.